In [ ]:
import pandas as pd
import numpy as np  
from lib.question_type_recommender import question_type_recommender

from lib.picklist_recommender import picklist_recommender
from lib.issue_parser import issue_parser 
from lib.questionnaire_parser import questionnaire_parser
from lib.dbupdate_parser import dbupdate_parser
from lib.script_generator import script_generator 
from lib.db_parser import db_parser  

import re, json, logging, requests, random 
from lib.context import context
from lib.utils import *   
from lib.script_generator import script_generator 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
qr = question_type_recommender(ctx, verbose=True, use_cache=True)
qr.recommend('If Yes to 1B 17.0,  what is the time requirement for reporting? ')

In [ ]:
qt = sql_todf("SELECT CAST(PK_QuestionTypeId AS NVARCHAR(99)) as [{PK_QuestionType}], code AS [{QT_CODE}] FROM fsma_QuestionTypes WHERE code IN ('LABEL','YN','CNT','TXT','PICK','FREQ','DEC','MULTICHECKBOX')",  ctx.connstr) 
qt

In [ ]:
parser = db_parser(ctx)
df24 = parser.parse(question_group='5141')
df24['{PK_PickListType}'] = df24['{PK_PickListType}'].apply(lambda s: str(s).replace('None','')).astype(str) 
df24['{PK_Question24}'] = df24['{PK_Question}'].apply(lambda s: int(s)).astype(str)  
df24['{FK_QuestionType}'] = df24['{FK_QuestionType}'].astype(str) 
df24 = df24[['{IdText}', '{FK_QuestionType}', '{PK_Question24}']]
df24[15:19]

In [ ]:
df24 = parser.parse(question_group='5140:5146') 
df24['{PK_PickListType}'] = df24['{PK_PickListType}'].apply(lambda s: str(s).replace('None','')).astype(str) 
df24['{PK_Question24}'] = df24['{PK_Question}'].apply(lambda s: int(s)).astype(str)  
df24['{PK_QuestionGroup24}'] = df24['{PK_QuestionGroup}'].apply(lambda s: int(s)).astype(str) 
df24['{sortpos24}'] = df24['{sortpos}'].apply(lambda s: int(s)) 
df24['{FK_QuestionType}'] = df24['{FK_QuestionType}'].astype(str) 
df24 = df24[['{IdText}', '{FK_QuestionType}', '{PK_Question24}', '{PK_QuestionGroup24}', '{sortpos24}']]
df24 = df24.sort_values(by=['{PK_QuestionGroup24}', '{sortpos24}'])
df24 

,{IdText},{FK_QuestionType},{PK_Question24},{PK_QuestionGroup24},{sortpos24}
1,.0,18,64500,5140,1
2,.0,18,64501,5140,1
8,1.0,9,64502,5140,2
17,2.0,9,64503,5140,3
25,3.0,9,64504,5140,4
33,4.0,17,64505,5140,5
40,4.1,9,64506,5140,6
43,5.0,17,64507,5140,7
46,6.0,17,64508,5140,8
49,7.0,1,64509,5140,9


In [ ]:
plr=picklist_recommender(ctx, use_cache=True)

In [ ]:
s="""Environmental
Financial 
Safety
Public Trust or Reputation
Operational 
Other""".split('\n')
s

rec = plr.recommend(s, generate_sql=False, threshhold=(.825, .7))
rec

In [ ]:
qr = question_type_recommender(ctx, verbose=True, use_cache=True)
 
def hvaprocess( df25 ):
    df25 = df25.fillna('') 
    df25 = df25.loc[ df25['{FY25}'] != 'Deleted' ]  
    df25['{FY24}'] = [ q[1] if re.match('.*New.*', q[0] ) else q[0] for q in zip( df25['{FY24}'], df25['{FY25}'] )  ]  
    df25['{QuestionText}'] =  df25['{QuestionText}'].apply(lambda s: re.sub('\n|\r',r'', s) )
    df25['{QuestionText}'] =  df25['{QuestionText}'].apply(lambda s: s.replace("'","''")) 
    df25['{QuestionText}'] = [ q[0] if re.match('.*[a-z].*', q[0] ) else q[1] for q in zip( df25['{FY24}'], df25['{QuestionText}'] )  ]   
    df25 = df25[ df25['{QuestionText}'] != '' ]  
    
    df25['{sortpos}'] = df25.index + 1
    df25['{PK_Question}'] = (df25['{sortpos}']+72000).astype(str) 
    df25['{idt}'] = df25['{FY25}']
    df25['{PK_QuestionType}'] = '9' 
    df25['{dep}'] = '' 
    df25['{FK_Question_Master}'] = ''
    df25['{PK_PickListType}'] = ''
    for i,r in df25.iterrows():     
        fmt = r['{fmt}']
        m = re.match('(.*):(.*)', fmt, re.DOTALL) 
        if m: 
            pl = [p.strip() for p in re.sub('\n\n','\n',m.groups(0)[1]).split('\n')  if p.strip() != ''  ] 
            rec = plr.recommend(pl, generate_sql=False, threshhold=(.825, .7))  
            df25.loc[i, '{PK_PickListType}']=str(rec['PK_PicklistType'])

        df25.loc[i, '{PK_QuestionType}']=qr.recommend(r['{QuestionText}'])['PK_QuestionType'].astype(str) 
        dep = re.match('.*If.*(1B\s[\.|\d]+)\s?.*', r['{QuestionText}'] ) 
        if(dep): 
            depid =dep.groups(0)[0] 
            df25.loc[i, '{dep}'] = depid
            try:
                pkdep = df24.loc[ df24['{IdText}'] == depid]['{PK_Question24}'].values[0] 
                df25.loc[i, '{FK_Question_Master}'] = pkdep 
            except:
                pass
                
    df25['{PK_QuestionType}'] = ['17' if re.match('.*Picklist.*',q[0])  else q[1] for q in zip(df25['{fmt}'], df25['{PK_QuestionType}'])]
    df25['{PK_QuestionType}'] = ['17' if re.match('.*Select One.*',q[0])  else q[1] for q in zip(df25['{fmt}'], df25['{PK_QuestionType}'])] 
    df25['{PK_QuestionType}'] = ['43' if re.match('.*Select.*Apply.*',q[0])  else q[1] for q in zip(df25['{fmt}'], df25['{PK_QuestionType}'])]
    df25['{PK_QuestionType}'] = ['2' if re.match('.*Integer.*',q[0])  else q[1] for q in zip(df25['{fmt}'], df25['{PK_QuestionType}'])]
    df25['{PK_QuestionType}'] = ['1' if re.match('.*Yes.*No.*',q[0])  else q[1] for q in zip(df25['{fmt}'], df25['{PK_QuestionType}'])]
    df25['{PK_PickListType}'] = ['NULL' if re.match('.*Yes.*No.*',q[0])  else q[1] for q in zip(df25['{fmt}'], df25['{PK_PickListType}'])]
    df25['{PK_PickListType}']=df25['{PK_PickListType}'].fillna('')
    df25['{ro}'] = df25['{ro}'].apply(lambda s: re.sub('\n|\r','',s))
    return df25
 
file = fr'{ctx.get_download_path()}\hva\1B.xlsx' 
df25 = pd.read_excel(file, header=0, sheet_name='Sheet1',  usecols='A:Z', dtype=str)  

df25 = hvaprocess(df25)
df25['{FY24}'] = df25['{FY24}'].apply(lambda s: re.sub('B(\d)',r"B \1", str(s)) )
df25['{FY24}'] = df25['{FY24}'].apply(lambda s: str(s) + '.0'  if '.' not in s else s)  
df25['{FY24}'] = df25['{FY24}'].apply(lambda s: s if re.match('^1B.*', s) else '0')  
df25['{FY25}'] = [ q[0] if re.match('^0.*', q[0] ) else q[1] for q in zip( df25['{FY24}'], df25['{FY25}'] )  ]
df25['{PK_QuestionType}'] = ['18' if re.match('^0',q[0])  else q[1] for q in zip(df25['{FY24}'], df25['{PK_QuestionType}'])]
df25 = pd.merge(df25, qt, how='left', left_on='{PK_QuestionType}', right_on='{PK_QuestionType}')
  
gen=script_generator(ctx)
code=gen.generate( df25 , code_template_path=f'{ctx.get_tempalte_dir()}hva_insert.sql')   
with open(ctx.get_dest()+'script.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{code}' )  
form_code = gen.generate(df25)  
with open(fr"{ctx.get_dest()}\script.aspx", 'w', encoding='UTF-8') as fw: 
    form_code =   form_code.replace('\ufeff', '')
    fw.write(form_code)

df25[21:26] 


In [ ]:
merged = pd.merge(df25, df24, how='left', left_on='{FY24}', right_on='{IdText}', suffixes=('','_24') ) 
merged['{idt}'] = merged['{FY25}'].apply(lambda s: re.sub('\.|\s','_',str(s)))
merged[8:11]     

In [ ]:
gen=script_generator(ctx)
code=gen.generate( merged , code_template_path=f'{ctx.get_tempalte_dir()}hva_insert.sql')  
# print( code ) 
with open(ctx.get_dest()+'script.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{code}' ) 

gen=script_generator(ctx)
form_code = gen.generate(merged)  
with open(fr"{ctx.get_dest()}\script.aspx", 'w', encoding='UTF-8') as fw: 
    form_code =   form_code.replace('\ufeff', '')
    fw.write(form_code)

In [ ]:
merged




In [ ]:
for i, r in merged.iterrows(): 
    if 'picklist' in r['{change}'].lower(): 
        lst = re.sub('Select all that Apply:\s*\n|Select One:\s*\n', '', r['{fmt}'], flags=re.IGNORECASE) 
        lst=lst.split('\n')  
        pl = picklist_recommender(ctx, use_cache=False).recommend(lst, generate_sql=True, threshhold=(1.825, 1.7))
        pl

In [ ]:
deps = merged[:] #.loc[ merged['{dep}'] != '' ]
deps = deps[['{FY25}','{PK_Question}','{FY24}','{PK_Question24}','{dep}','{QuestionText}'  ]]
deps[44:48] 

In [ ]:
df2425 = merged[['FY24ID','FY25ID','']]

In [ ]:
qr = question_type_recommender( ctx, verbose=True, use_cache=True)

In [ ]:
picklist_string = 'Date Determined / TBD / Continuous ATO'  
pl=picklist_recommender(ctx, use_cache=False).recommend(picklist_string.split(' / '), generate_sql=True, threshhold=(.825, .7))
pl 

In [ ]:
file=r'C:\Users\timko\Downloads\Untitled.txt'
txt = ''
df=[]

with open(file, 'r') as f:
    txt= f.read()
    txt = txt.split('\n')
for t in txt:
    g=re.match('(\d\.\d\.?\d?\.?\d?\.?\d?\.?)\s(.+)',t, re.DOTALL)
    if g: 
     FK_QuestionType = qr.recommend( g.groups(0)[1] )['PK_QuestionType']
     #if FK_QuestionType != 2: FK_QuestionType = 18
     df.append({'{PK_Question}':'00000', '{IdText}':g.groups(0)[0], '{QText}':g.groups(0)[1], '{FK_QuestionType}':FK_QuestionType  })
     #print (g.groups(0) )
df=pd.DataFrame(df)
df[90:]

In [ ]:
gen=script_generator(ctx)
code=gen.generate(df, code_template_path=f'{ctx.get_tempalte_dir()}cio_question_insert.sql')  
print( code ) 

In [ ]:
config = {}
with open('C:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope.Tests\AutomatorConfig.json', 'rb') as f: 
    config=json.loads(f.read())    
dic = config['InputDefaults']["//form[contains(@action, '_CIO_')]"]

lod=[]
for k in dic.keys():
    lod.append({'regex':k, 'value':dic[k]})
df=pd.DataFrame(lod)
df.to_csv( r"C:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope.Tests\Data\form[contains(@action, '_CIO_')].csv" , index=False)


In [ ]:
pk = sql_todf("SELECT DisplayValue, PK_Picklist FROM vwPicklists WHERE UsageField='ServiceModelType' ", config['connstr'])
pk

In [ ]:
ep = sql_todf("""
SELECT
	  REPLACE(p.name,'cq_','') AS [spname]
    , pa.parameter_id AS [order]
    , pa.name AS [name]
    , UPPER(t.name) AS [type]
    , t.max_length AS [length]  
FROM sys.parameters AS pa 
INNER JOIN sys.procedures AS p on pa.object_id = p.object_id 
INNER JOIN sys.types AS t on pa.system_type_id = t.system_type_id AND pa.user_type_id = t.user_type_id
WHERE 1=1
AND create_date > '06-06-2024'
AND p.name LIKE  'cq_%'

""", config['connstr'])
ep



In [ ]:
ep_g = ep[['spname','name']].groupby('spname', as_index=False).agg({'name':list})
ep_g

In [ ]:
def to_routes(lst):
    s= map( lambda s: '[FromQuery] string? '+s.replace('@','')+', ' , lst) 
    return ''.join(list(s))
to_routes(['@PK_ReportingCycle','@ComponentGroup'])


In [ ]:
def to_params(lst): 
    rq,p='','' 
    for l in lst:
        l= re.sub('@','',l)
        #rq=rq+f'\n    var {l} = request.Query["{l}"] ?? ""; '
        p=p+f'\n    request.PARMS.Add("@{l}", {l}); '
    return rq+''+p
s = to_params(['@Generic1','@PK_ReportingCycle'])
print(s)

In [ ]:
snip=''
with open(ctx.get_tempalte_dir()+ 'snip.cs', 'r', encoding='UTF-8') as f: 
    snip=f.read() 
s=''
for i,r in ep_g.iterrows():
    args = to_routes(r['name'])
    #print(params) 
    s=s+snip.replace('~args~', args).replace('~cq~', r['spname']).replace('~parms~', to_params(r['name']))

print(s)

with open(ctx.get_dest()+'script.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{s}' ) 

In [ ]:
dest=''
with open(r'C:\dev\CyberScope\CsLab\Cyberscope\CSwebdev\code\CSAPI\Program.cs', 'r', encoding='UTF-8') as f: 
    dest=f.read() 
m=re.search('.*(MapGet.+)app\.Run',dest, flags=re.DOTALL)
m


In [ ]:
file=fr'{ctx.get_download_path()}\USDA-BOD19-02Vulnerabilities.xlsx'  
df=pd.read_excel(file, header=0, sheet_name='Sheet1',  usecols='A:Z', dtype=str) 
df=df.fillna('')  
df 


In [ ]:
sql = ''
for i  in range(0,len(df)) : 
    vals = "'"+ "',N'".join( list(df.loc[i].values )) + "'"
    vals = vals + ',@PK_OrgSubmission,@PK_Component,0,' + str(i+1)
    vals = vals.replace('1900-01-01 00:00:00', '')
    vals =  'insert into @p2 values( ' + vals + ') '
    sql= sql + vals + '\n' 

script='' 
with open(ctx.get_down_dir() + '1902_hhs_upload.sql', 'r', encoding='UTF-8') as f: 
    script=f.read()

script=script.replace('RETURN;', sql + '\nRETURN;\n' )
 
with open(ctx.get_dest()+'script.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{script}' ) 




In [ ]:
aspx=''
for i in range(1,8):
    aspx=aspx+f'<tr>\n'
    aspx=aspx+ '<td class="workroleid">' + f'<uc:CBtext2 ID="CBtext_6_1_{i}" QuestionNumber="6.1.{i}" runat="server" CSSClass="CustomControlValue " Columns="1" Rows="1" />' + '</td> \n'  
    for l in ['a','b','c']:
        s=''
        s=s+f'<uc:CBnumeric2 ID="CBNumeric_6_1_{i}_{l}" runat="server" CSSClass="CustomControlValue" QuestionNumber="6.1.{i}.{l}" /> '
        s=s+f'<uc:CBoptionalLinks ID="CBoptionalLinks_6_1_{i}_{l}" LinkEditID="CBNumeric_6_1_{i}_{l}" runat="server"  ShowAudit="True" ShowHelp="True" ShowNarrative="true" AuditRecs="20" />' 
        aspx=aspx+ '<td>' + s  + '</td> \n' 
    aspx=aspx+'</tr> \n'
print ( aspx )

In [ ]:
sql = SQL_INSERT_FROM_DF(df, 'NICEWorkRoles')
s = '\n'.join( sql[0] ) 
with open(ctx.get_dest()+'script.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{s}' ) 

In [ ]:

s=''
for i,r in df.iterrows(): 

    s=s+f' UPDATE FEDRAMP_UPLOAD SET ServiceModel=' + str(r["PK_Picklist"]) + ' WHERE FEDRAMPID=\'' + r["FedRAMP ID"] + '\';\n'

with open(ctx.get_dest()+'script.js', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{s}' ) 

print( s )


In [ ]:
# AGENCY, BUREAU, FedRAMPPackageID, FedRAMPATOIssuanceDate, FedRAMPATOExpirationDate
s=''
for i,r in df.iterrows():

    s = s + f"\n('{r['Parent Agency']}', '{r['Sub Agency']}', '{r['FedRAMP ID']}', '{r['ATO Issuance Date']}', '{r['ATO Expiration Date']}'),"
s=s.replace(' 00:00:00','')
s=s.replace("'nan'",'NULL')

s=s[:-1]
with open(ctx.get_dest()+'script.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{s}' ) 


In [ ]:
df2023 = sql_todf("  SELECT Idtext, CAST(PK_Question  AS NVARCHAR(9)) AS PK_23, CAST(Q_TypeID AS NVARCHAR(9)) AS Q_Type_23, CAST(FK_PicklistType AS NVARCHAR(9)) AS FK_PicklistType_23 FROM vwQuestions WHERE PK_Form='2023-A-HVA' AND Q_TypeID NOT IN (18, 92, 63, 69, 70)  ", config['connstr'])
df2024 = sql_todf("  SELECT Idtext, CAST(PK_Question  AS NVARCHAR(9)) AS PK_24, CAST(Q_TypeID AS NVARCHAR(9)) AS Q_Type_24, CAST(FK_PicklistType AS NVARCHAR(9)) AS FK_PicklistType_24 FROM vwQuestions WHERE PK_Form='2024-A-HVA' AND Q_TypeID NOT IN (18, 92, 63, 69, 70)  ", config['connstr'])
merged = df2023.merge(map, left_on='Idtext', right_on='FY23 #', how='left')
merged = df2024.merge(merged, left_on='Idtext', right_on='FY24 #', how='left')
merged=merged.fillna('-')
merged[90:125]

In [ ]:
df = sql_todf(" SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME ='HVAEXT2'  ", config['connstr'])
l=[]
for i,r in df.iterrows():
    l.append(r['COLUMN_NAME'])
print(','.join(l))

In [ ]:
lod=[]
for i in range(0,len(df2024)-1): 
    input=normalize(df2024.iloc[i]['QTEXT'])
    Idtext=df2024.iloc[i]['Idtext']
    for ii,r in df2023[:].iterrows():
        test=normalize(r['QTEXT'] ) 
        if 'PLEASE DESCRIBE' in test:
            continue
        sequence_match = SequenceMatcher(None, test,input).ratio()
        if sequence_match > .75 and Idtext != r['Idtext'] :
            print(r['Idtext'] + ' -> ' + Idtext + ' :' + test)
            lod.append({
                'Idtext_from':r['Idtext'],
                'Idtext_to':Idtext,
                'text':test,
                'sm':sequence_match 
            })

df=pd.DataFrame(lod)

In [ ]:

s='' 
for i,r in df.iterrows(): 
    m23="'"+str(r["FY23 #"])+"'"
    m24="'"+str(r["FY24 #"])+"'"
    s = s + f'( {m23}, {m24} ), \n' 
print( s ) 

In [ ]:
t = zip( list( df['FY23 #'] ), list( df['FY24 #'] ))
output = "".join(t)
output
 

In [ ]:
s1="""		
    UPDATE I SET I.AnsPK_{c} = ANS.AnsPK_{c} FROM ##CIO_CDM_IMPORT I 
    INNER JOIN #CIO_CDM_Data_TMP ANS ON ANS.PK_OrgSubmission = I.PK_OrgSubmission AND ANS.QPK_{c} = I.QPK_{c}"""
s2="""
		UPDATE A Set Answer = CAST(CAST({c} AS DECIMAL(8,2)) AS NVARCHAR(999)) FROM fsma_Answers A INNER JOIN ##CIO_CDM_IMPORT I ON I.AnsPK_{c}=A.PK_Answer ; """
 
s3="""
		SELECT QPK_{c}, PK_OrgSubmission, CAST(CAST({c} AS DECIMAL(8,2)) AS NVARCHAR(999)), GETDATE() FROM ##CIO_CDM_IMPORT WHERE AnsPK_{c} IS NULL
		UNION """
s4="""
			INSERT INTO AuditLog (TableName, PK_PrimeKey, FieldName, FieldValue, Change_Date, UserID, EditType, String_PrimeKey)  
			SELECT 'fsma_Answers', FK_OrgSubmission, QPK_{c}, Answer, @GETDATE, 0, 'D', CAST(PK_Answer as NVARCHAR(99))
			FROM fsma_Answers A
			INNER JOIN ##CIO_CDM_IMPORT I ON I.PK_OrgSubmission=A.FK_OrgSubmission
			AND FK_Question=QPK_{c} 
            """
s5="""          
                --{c}
				SELECT @FK_Question= QPK_{c}, @Answer={c}, @PK_OrgSubmission=PK_OrgSubmission FROM ##CIO_CDM_IMPORT WHERE ROWNUM=@i; 
				SELECT TOP 1 @PK_Answer=PK_Answer FROM #temp_Answers WHERE FK_Question=@FK_Question AND FK_OrgSubmission=@PK_OrgSubmission 
				IF @PK_Answer IS NOT NULL
					BEGIN
						UPDATE fsma_Answers SET Answer=@Answer, DateModifed=@GETDATE WHERE PK_Answer=@PK_Answer
					END
				ELSE
					BEGIN
						INSERT INTO fsma_Answers (FK_Question, FK_OrgSubmission, Answer, DateModifed)
						SELECT QPK_{c}, PK_OrgSubmission, GFE_endpoints, @GETDATE
						FROM ##CIO_CDM_IMPORT WHERE ROWNUM=@i;
					END
"""
lst=['GFE_endpoints','GFE_networking','GFE_inout','GFE_unknown','cisa_7days','cisa_14days','cisa_30days']
lst = [s5.replace('{c}',i)  for i in lst]
print(''.join(lst))

In [ ]:
s = ', '
c =  [ f", @{re.sub('[^A-Za-z0-9]','', c)} NVARCHAR(99)" for c in df.columns ]
print( '\n'.join(c) )

In [ ]:
def formatqt(s):
    s=s.replace('•','') 
    s=s.replace('','<br>')
    s=re.sub('[\n|\r]','',s) 
    return s  
l=formatqt(""" """) 
insert='\nINSERT INTO fsma_Questions (PK_Question, FormName, FK_QuestionGroup,sortpos,FK_PickListType, FK_QuestionType,identifier_text,ExternalLinkType, QuestionText, help_text) VALUES 	\n'

In [ ]:
file=f'{ctx.get_down_dir()}ig.xlsx' 
converts={'Question':lambda x: re.sub('\n','',x) } 
dff=pd.read_excel(file, converters=converts, sheet_name=int(0), header=0, usecols='A,P,Q,R,X,W,Y,U,V,AB,AC,AD,AE,AF,AI,AJ,AK,AL,AO,AQ,AP,AR' ) 
dff[106:110]

In [ ]:
df=dff 
df = df[df["Question"].str.contains("Question") == False]
df["{QuestionText}"]=df["Question"]
 
df=df.fillna('').astype(str) 
 
df['ht']=df['Unnamed: 15']+df['Unnamed: 17']+df['Maturity Level'] 
df['ht'].fillna('', inplace=True)
df['{helptext}']=df['ht'].apply(lambda s: f'<li> <strong>Ad Hoc:</strong>{s}</li>') 
df.drop(columns=['Unnamed: 15','Unnamed: 17','Maturity Level'], inplace=True) 

df['ht']=df['Unnamed: 20']+df['Unnamed: 21']+ df['Unnamed: 22'] + df['Unnamed: 23']+df['Unnamed: 24'] 
df['ht'].fillna('', inplace=True)
df['{helptext}']=df['{helptext}']+df['ht'].apply(lambda s: f'<li> <strong>Defined:</strong>{s}</li>')
df.drop(columns=['Unnamed: 20', 'Unnamed: 21','Unnamed: 22', 'Unnamed: 23','Unnamed: 24'], inplace=True)

df['ht']=df['Unnamed: 27']+df['Unnamed: 28']+df['Unnamed: 29'] + df['Unnamed: 30']+df['Unnamed: 31']
df['ht'].fillna('', inplace=True)
df['{helptext}']=df['{helptext}']+df['ht'].apply(lambda s: f'<li> <strong>Consistently Implemented:</strong>{s}</li>')
df.drop(columns=['Unnamed: 27','Unnamed: 28','Unnamed: 29','Unnamed: 30','Unnamed: 31'], inplace=True)

df['ht']=df['Unnamed: 34']+ df['Unnamed: 35']+ df['Unnamed: 36'] +df['Unnamed: 37']
df['ht'].fillna('', inplace=True)
df['{helptext}']=df['{helptext}']+df['ht'].apply(lambda s: f'<li> <strong>Managed and Measurable:</strong>{s}</li>')
df.drop(columns=['Unnamed: 34', 'Unnamed: 35','Unnamed: 36','Unnamed: 37'], inplace=True)

df['ht']=df['Unnamed: 40']+df['Unnamed: 41']+df['Unnamed: 42']+df['Unnamed: 43'] 
df['ht'].fillna('', inplace=True)
df['{helptext}']=df['{helptext}']+df['ht'].apply(lambda s: f'<li> <strong>Optimized:</strong>{s}</li>')
df.drop(columns=['Unnamed: 40','Unnamed: 41','Unnamed: 42','Unnamed: 43'], inplace=True)
 
df.drop(columns=['ht', 'Question'], inplace=True)

df['{helptext}']=df['{helptext}'].apply(lambda s: f'<br/><br/><h6><b>Maturity Level:</b></h6><ul>{s}</ul>' if len(s) > 210 else '' )

df['{helptext}']=df['{helptext}'].apply(formatqt)
df['{QuestionText}']=df['{QuestionText}'].apply(formatqt)
df['{idt}']=df['{QuestionText}'].apply(lambda s: re.findall('(\d{1,2}.\d?\s{1,4})',s)[0].rstrip(" ").rstrip(".") )
df['{FK_PickListType}']='85' 
df['{FK_QuestionType}']=df['{QuestionText}'].apply(lambda s: '9' if "additional information" in s  else '17')

df['{QuestionText}']=df['{QuestionText}'].apply(lambda s: re.sub('\'','`',s) )
df['{helptext}']=df['{helptext}'].apply(lambda s: re.sub('\'','`',s) )
s=2
sort=0
pk=53009
for i,r, in df.iterrows():
    if re.search('Table \d{1,2}',r['{QuestionText}']):
        s=s+1
        sort=0
        pk=pk+10 
    else:
        if sort > 0:
            df.loc[i, '{pre}']=str('\n,')
        else:
            df.loc[i, '{pre}']=str(insert)
        sort=sort+1
        pk=pk+1
        df.loc[i, '{FK_QuestionGroup}']=f'@PK_QGroup+{s}'
        df.loc[i, '{sortpos}']=str(sort)
        df.loc[i, '{PK_Question}']=str(pk) 
df = df[df["{QuestionText}"].str.contains("Table ") == False]
df['{QuestionText}']=df['{QuestionText}'].apply(lambda s: re.sub('\d{1,2}.\d?\s{1,4}','',s).strip(" ") ) 
df[['{QuestionText}','{helptext}']].to_html('out/script.html', escape=False)


In [ ]:
df

In [ ]:
gen=script_generator(ctx)
code=gen.generate(df, code_template_path=f'{ctx.get_tempalte_dir()}fsma_QuestionsInsert.sql')  

In [ ]:
with open(ctx.get_dest()+'script.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{code}' ) 

In [ ]:
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\_5_DBUpdate7.53_IG.sql', 'r', encoding='UTF-8') as f: 
    sql=f.read() 
m=re.search('--PARSEBEGIN(.*)--PARSEEND',sql, flags=re.DOTALL)
sql=sql.replace(m.group(1),code)
with open(ctx.get_dest()+'script.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{sql}' ) 

#with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\_5_DBUpdate7.53_IG.sql', 'w', encoding='UTF-8') as f: 
#    f.write() 
